<a href="https://colab.research.google.com/github/Ini-design/GenAi/blob/main/AiExplain17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers torch gradio

import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load a free lightweight model (works without API key)
model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="cpu"   # change to "cuda" if GPU is available
)

# Function for explaining a topic
def explain_topic(topic):
    prompt = f"Explain the following topic clearly and simply:\n\nTopic: {topic}\n\nExplanation:"

    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.4,
        do_sample=True
    )

    explanation = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the prompt part
    if "Explanation:" in explanation:
        explanation = explanation.split("Explanation:")[-1].strip()

    return explanation

# Build Gradio interface
interface = gr.Interface(
    fn=explain_topic,
    inputs=gr.Textbox(label="Enter a topic you want explained"),
    outputs=gr.Textbox(label="Explanation"),
    title="AI Explainer",
    description="Enter any topic and the AI will explain it clearly."
)

interface.launch(share=True)